In [1]:

import pandas as pd
from pathlib import Path

excel_path = Path(r"/Users/cheickberthe/PycharmProjects/spreadsheet-analyzer/test_assets/collection/business-accounting/Business Accounting.xlsx")
df = pd.read_excel(excel_path, sheet_name=0)
print(f"Loaded sheet with shape: {df.shape}")

# Quick preview for LLM context
print("\nFirst 10 rows:")
print(df.head(10).to_string())
print("\nLast 5 rows:")
print(df.tail(5).to_string())

# Get basic info
sheet_name = "None"
sheet_dimensions = f"{df.shape[0]} rows x {df.shape[1]} columns"


Loaded sheet with shape: (10, 9)

First 10 rows:
        Date                                   Description   USD Amount  Transaction type   Category   TOTAL REVENUES   0   TOTAL EXPENSES  -2031.63
0 2025-01-03                  Mali Apartment work - Cement       -43.51               EXP        NaN              NaN NaN              NaN       NaN
1 2025-01-15            Rebtel International Phone charges        -9.49               EXP        NaN              NaN NaN              NaN       NaN
2 2025-01-21                  Mali Apartment work - Cement        -8.57               EXP        NaN              NaN NaN              NaN       NaN
3 2025-01-22  Mali Apartment Work - Tile Install (Kitchen)       -48.78               EXP        NaN              NaN NaN              NaN       NaN
4 2025-01-23              Aluminum fabricator - Issa Maiga      -424.34               EXP        NaN              NaN NaN              NaN       NaN
5 2025-01-23  Mali Apartment work - Kitchen Wall Crepisag

In [2]:

excel_file_name = excel_path.name
sheet_dimensions = f"{df.shape[0]} rows x {df.shape[1]} columns"
sheet_name


'None'

In [3]:

# Table Detection Analysis
print("=== DATA STRUCTURE ANALYSIS ===")
print(f"DataFrame shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\n=== FIRST 10 ROWS ===")
print(df.head(10).to_string())
print("\n=== LAST 5 ROWS ===")
print(df.tail(5).to_string())

print("\n=== COLUMN ANALYSIS ===")
# Check for empty columns that might separate tables
for i, col in enumerate(df.columns):
    non_null_count = df[col].notna().sum()
    print(f"Column {i} ('{col}'): {non_null_count}/{len(df)} non-null values")

print("\n=== EMPTY ROW ANALYSIS ===")
# Check for empty rows that might separate tables
empty_rows = df.isnull().all(axis=1)
if empty_rows.any():
    empty_row_indices = empty_rows[empty_rows].index.tolist()
    print(f"Empty rows found at indices: {empty_row_indices[:10]}")
else:
    print("No completely empty rows found")

print("\n=== SIDE-BY-SIDE ANALYSIS ===")
# Check for potential side-by-side tables by examining column groups
left_cols = df.iloc[:, 0:7]  # First 7 columns
right_cols = df.iloc[:, 7:]  # Remaining columns
print(f"Left section shape: {left_cols.shape}")
print(f"Right section shape: {right_cols.shape}")
print(f"Left section non-null density: {left_cols.notna().sum().sum() / (left_cols.shape[0] * left_cols.shape[1]):.2f}")
print(f"Right section non-null density: {right_cols.notna().sum().sum() / (right_cols.shape[0] * right_cols.shape[1]):.2f}")


=== DATA STRUCTURE ANALYSIS ===
DataFrame shape: (10, 9)
Columns: ['Date', 'Description', 'USD Amount', 'Transaction type', 'Category', 'TOTAL REVENUES', 0, 'TOTAL EXPENSES', -2031.6299999999999]

=== FIRST 10 ROWS ===
        Date                                   Description   USD Amount  Transaction type   Category   TOTAL REVENUES   0   TOTAL EXPENSES  -2031.63
0 2025-01-03                  Mali Apartment work - Cement       -43.51               EXP        NaN              NaN NaN              NaN       NaN
1 2025-01-15            Rebtel International Phone charges        -9.49               EXP        NaN              NaN NaN              NaN       NaN
2 2025-01-21                  Mali Apartment work - Cement        -8.57               EXP        NaN              NaN NaN              NaN       NaN
3 2025-01-22  Mali Apartment Work - Tile Install (Kitchen)       -48.78               EXP        NaN              NaN NaN              NaN       NaN
4 2025-01-23              Aluminum f

In [4]:
# Create detected_tables based on the data structure
detected_tables = [
    {
        "table_id": "transaction_log",
        "description": "Expense transactions for January 2025",
        "start_row": 0,
        "end_row": 9,
        "start_col": 0,
        "end_col": 4,
        "confidence": 0.95,
        "table_type": "DETAIL",
        "entity_type": "expenses"
    },
    {
        "table_id": "transaction_summary",
        "description": "Transaction total summary columns",
        "start_row": 0,
        "end_row": 9,
        "start_col": 5,
        "end_col": 8,
        "confidence": 0.3,  # Low confidence due to lack of data
        "table_type": "SUMMARY",
        "entity_type": "totals"
    }
]

print(f"Detected {len(detected_tables)} tables")
for i, table in enumerate(detected_tables, 1):
    print(f"Table {i}: {table['description']} at rows {table['start_row']}-{table['end_row']}, cols {table['start_col']}-{table['end_col']}")

Detected 2 tables
Table 1: Expense transactions for January 2025 at rows 0-9, cols 0-4
Table 2: Transaction total summary columns at rows 0-9, cols 5-8


In [5]:

# Extract detection results created by LLM
# The detector should have created a 'detected_tables' variable with the results

# CLAUDE-TEST-WORKAROUND: Validate that detected_tables exists and is properly formatted
if 'detected_tables' in globals():
    # Validate it's a list
    if isinstance(detected_tables, list):
        detection_results = detected_tables
        print(f"✅ Found {len(detection_results)} tables from LLM detection")
        # Validate first table has required fields (if any tables exist)
        if detection_results:
            required_fields = ['table_id', 'description', 'start_row', 'end_row', 'start_col', 'end_col']
            first_table = detection_results[0]
            missing_fields = [f for f in required_fields if f not in first_table]
            if missing_fields:
                print(f"⚠️ Warning: First table missing fields: {missing_fields}")
    else:
        print(f"❌ Error: detected_tables is not a list, it's a {type(detected_tables)}")
        detection_results = []
else:
    # CLAUDE-GOTCHA: Gemini sometimes fails to create the variable even after multiple prompts
    print("❌ No 'detected_tables' variable found - LLM failed to complete detection")
    detection_results = []

detection_results


✅ Found 2 tables from LLM detection


[{'table_id': 'transaction_log',
  'description': 'Expense transactions for January 2025',
  'start_row': 0,
  'end_row': 9,
  'start_col': 0,
  'end_col': 4,
  'confidence': 0.95,
  'table_type': 'DETAIL',
  'entity_type': 'expenses'},
 {'table_id': 'transaction_summary',
  'description': 'Transaction total summary columns',
  'start_row': 0,
  'end_row': 9,
  'start_col': 5,
  'end_col': 8,
  'confidence': 0.3,
  'table_type': 'SUMMARY',
  'entity_type': 'totals'}]


# Table Detection Results

Detected 2 tables:


## Table 1: Expense transactions for January 2025
- Location: Rows 0-9, Columns 0-4
- Type: detail
- Entity: expenses
- Confidence: 0.95

## Table 2: Transaction total summary columns
- Location: Rows 0-9, Columns 5-8
- Type: summary
- Entity: totals
- Confidence: 0.30
